In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import datetime


In [62]:
''' 

We want to extract these data from the dataframe: 
For each maze we want to know: 
- first choice
- how much consecutive time they spend with one object  before moving (define a radius for the distance after what we consider the fly to be away )
- sum of the consectuive times with an object = total time 
- how many time they go to an object 
- total time without interacting 
- tot time with reinforced object 
for unused Na 

'''

' \n\nWe want to extract these data from the dataframe: \nFor each maze we want to know: \n- first choice\n- how much consecutive time they spend with one object  before moving (define a radius for the distance after what we consider the fly to be away )\n- sum of the consectuive times with an object = total time \n- how many time they go to an object \n- total time without interacting \n- tot time with reinforced object \nfor unused Na \n\n'

In [66]:
#read the csv file saved here home/matthias/Videos/Alice_Samara_Videos2/ly_data_AliceSamara.csv and store it in a pentus dataframe
# Specify the path to the CSV file
file_path = "/home/matthias/Videos/Alice_Samara_Videos2/fly_data_AliceSamara.csv"

# Read the CSV file and store it in a DataFrame
flydata_AS_df = pd.read_csv(file_path)

#create a new datadrame with one row for each maze of each arena and keep the column about arena, exp, maze, starving_cond, habituation_cond
flydata_AS_resume_df = flydata_AS_df[['arena', 'exp', 'maze', 'starving_cond', 'habituation_cond']].drop_duplicates()



In [69]:
#drop dead flies from the flydata_AS_resume_df
# for each row in the flydata_AS_resume_df we need to analyse all the columns in the flydata_AS_df with the same arena, exp, maze, starving_cond, habituation_cond and find the total time the fly spent in each area
for index, row in flydata_AS_resume_df.iterrows():
    arena = row['arena']
    exp = row['exp']
    maze = row['maze']
    starving_cond = row['starving_cond']
    habituation_cond = row['habituation_cond']
    
    # Create a mask that selects rows where 'arena', 'exp', 'maze', 'starving_cond', and 'habituation_cond' match the current row
    
    mask = (flydata_AS_df['arena'] == arena) & (flydata_AS_df['exp'] == exp) & (flydata_AS_df['maze'] == maze) & (flydata_AS_df['starving_cond'] == starving_cond) & (flydata_AS_df['habituation_cond'] == habituation_cond)

    #check if the fly is dead
    #check if the values of head_x	head_y	thorax_x	thorax_y	abdo_x	abdo_y	are the same in all the rows of the mask (with a margin of 3)
    if (flydata_AS_df.loc[mask, 'head_x'].std() < 6) & (flydata_AS_df.loc[mask, 'thorax_x'].std() < 6) & (flydata_AS_df.loc[mask, 'abdo_x'].std() < 6):
        flydata_AS_resume_df.drop(index, inplace=True)


In [70]:
#initialize the columns we want to add to the flydata_AS_resume_df
flydata_AS_resume_df['first_choice'] = np.nan
flydata_AS_resume_df['tot_time_blue'] = np.nan
flydata_AS_resume_df['tot_time_oran'] = np.nan
flydata_AS_resume_df['tot_time_neutral'] = np.nan
flydata_AS_resume_df['rate_interactions_blue'] = np.nan
flydata_AS_resume_df['rate_interactions_oran'] = np.nan
flydata_AS_resume_df['max_consec_time_blue'] = np.nan
flydata_AS_resume_df['max_consec_time_oran'] = np.nan
flydata_AS_resume_df['tot_time_reinforc_obj'] = np.nan



In [71]:
#column first_choice
# for each row in the flydata_AS_resume_df we need to analyse all the columns in the flydata_AS_df with the same arena, exp, maze, starving_cond, habituation_cond and find the first choice of the fly
#the first choice of the fly is the first element of the column "area" different from "neutral"
#the first choice is stored in the column first_choice of the flydata_AS_resume_df
for index, row in flydata_AS_resume_df.iterrows():
    arena = row['arena']
    exp = row['exp']
    maze = row['maze']
    starving_cond = row['starving_cond']
    habituation_cond = row['habituation_cond']
    
    # Create a mask that selects rows where 'arena', 'exp', 'maze', 'starving_cond', and 'habituation_cond' match the current row and the area is not neutral
    mask = (flydata_AS_df['arena'] == arena) & (flydata_AS_df['area'] != 'neutral') & (flydata_AS_df['exp'] == exp) & (flydata_AS_df['maze'] == maze) & (flydata_AS_df['starving_cond'] == starving_cond) & (flydata_AS_df['habituation_cond'] == habituation_cond)

    #now within these rows we want to extract the value of the first element of the column "area"
    area_values = flydata_AS_df.loc[mask, 'area']
    
    if len(area_values) > 0:
        first_choice = area_values.iloc[0]
    else:
        first_choice = None  
        
    #we save it in the new df
    flydata_AS_resume_df.at[index, 'first_choice'] = first_choice

In [72]:
#column tot time
# for each row in the flydata_AS_resume_df we need to analyse all the columns in the flydata_AS_df with the same arena, exp, maze, starving_cond, habituation_cond and find the total time the fly spent in each area
for index, row in flydata_AS_resume_df.iterrows():
    arena = row['arena']
    exp = row['exp']
    maze = row['maze']
    starving_cond = row['starving_cond']
    habituation_cond = row['habituation_cond']
    
    # Create a mask that selects rows where 'arena', 'exp', 'maze', 'starving_cond', and 'habituation_cond' match the current row
    
    mask = (flydata_AS_df['arena'] == arena) & (flydata_AS_df['exp'] == exp) & (flydata_AS_df['maze'] == maze) & (flydata_AS_df['starving_cond'] == starving_cond) & (flydata_AS_df['habituation_cond'] == habituation_cond)

    #now within these rows we want to count the total number of rows with the area corresponding to orange, blue or neutral
    #we save it in the new df
    flydata_AS_resume_df.at[index, 'tot_time_blue'] =0.03448 * len(flydata_AS_df.loc[mask & (flydata_AS_df['area'] == 'blue')])
    flydata_AS_resume_df.at[index, 'tot_time_oran'] = 0.03448 * len(flydata_AS_df.loc[mask & (flydata_AS_df['area'] == 'orange')])
    flydata_AS_resume_df.at[index, 'tot_time_neutral'] = 0.03448 * len(flydata_AS_df.loc[mask & (flydata_AS_df['area'] == 'neutral')])
    

In [73]:
#column rate interactions

def count_consecutive_frames(df, area_value, min_length=58):
    count = 0
    current_streak = 0

    for value in df['area']:
        if value == area_value:
            current_streak += 1
            if current_streak == min_length:
                count += 1
        else:
            current_streak = 0

    return count


# for each row in the flydata_AS_resume_df we need to analyse all the columns in the flydata_AS_df with the same arena, exp, maze, starving_cond, habituation_cond 
for index, row in flydata_AS_resume_df.iterrows():
    arena = row['arena']
    exp = row['exp']
    maze = row['maze']
    starving_cond = row['starving_cond']
    habituation_cond = row['habituation_cond']
    
    # Create a mask that selects rows where 'arena', 'exp', 'maze', 'starving_cond', and 'habituation_cond' match the current row 
    mask = (flydata_AS_df['arena'] == arena) & (flydata_AS_df['exp'] == exp) & (flydata_AS_df['maze'] == maze) & (flydata_AS_df['starving_cond'] == starving_cond) & (flydata_AS_df['habituation_cond'] == habituation_cond)
    
    #within the rows of the mask in the flydata_AS_df we want to count how many times the fly stays in the orange or blue area for at least 2 seconds (58 frames)
    #to do that we need to count how many times there are at least 58 consecutive rows with the same area (orange or blue) and save this number in the new df
    
    filtered_df = flydata_AS_df[mask]

    # Count consecutive frames for 'orange' and 'blue'
    orange_count = count_consecutive_frames(filtered_df, 'orange')
    blue_count = count_consecutive_frames(filtered_df, 'blue')
    total_count = orange_count + blue_count
    #print(orange_count, blue_count, total_count)

    if total_count > 0:
        flydata_AS_resume_df.at[index, 'rate_interactions_blue'] = blue_count / total_count
        flydata_AS_resume_df.at[index, 'rate_interactions_oran'] = orange_count / total_count
    else:
        print('No interactions found for arena:', arena, 'exp:', exp, 'maze:', maze)



No interactions found for arena: 7 exp: 5 maze: 2
No interactions found for arena: 0 exp: 5 maze: 1
No interactions found for arena: 8 exp: 5 maze: 1
No interactions found for arena: 7 exp: 4 maze: 1


In [74]:
#max consecutive time in orange and blue
# for each row in the flydata_AS_resume_df we need to analyse all the columns in the flydata_AS_df with the same arena, exp, maze, starving_cond, habituation_cond and find the total time the fly spent in each area
for index, row in flydata_AS_resume_df.iterrows():
    arena = row['arena']
    exp = row['exp']
    maze = row['maze']
    starving_cond = row['starving_cond']
    habituation_cond = row['habituation_cond']
    
    # Create a mask that selects rows where 'arena', 'exp', 'maze', 'starving_cond', and 'habituation_cond' match the current row
    
    mask = (flydata_AS_df['arena'] == arena) & (flydata_AS_df['exp'] == exp) & (flydata_AS_df['maze'] == maze) & (flydata_AS_df['starving_cond'] == starving_cond) & (flydata_AS_df['habituation_cond'] == habituation_cond)

    #now within these rows we want to count how many consecutive columns with the same area orange or blue we have before the fly moves to another area and pick the maximum value of this count
    #we save it in the new df
    max_consec_time_blue = 0
    max_consec_time_orang = 0
    consec_time_blue = 0
    consec_time_orang = 0
    for i in range(len(flydata_AS_df.loc[mask])):
        if flydata_AS_df.loc[mask, 'area'].iloc[i] == 'blue':
            consec_time_blue += 1
            consec_time_orang = 0
        elif flydata_AS_df.loc[mask, 'area'].iloc[i] == 'orange':
            consec_time_orang += 1
            consec_time_blue = 0
        else:
            consec_time_orang = 0
            consec_time_blue = 0
        max_consec_time_blue = max(max_consec_time_blue, consec_time_blue)
        max_consec_time_orang = max(max_consec_time_orang, consec_time_orang)
    
    flydata_AS_resume_df.at[index, 'max_consec_time_blue'] = 0.03448 * max_consec_time_blue
    flydata_AS_resume_df.at[index, 'max_consec_time_oran'] = 0.03448 * max_consec_time_orang   
   

In [75]:
#total time with reinforced object 
#if habituation condition is blue then put in this column the value of tot_time_blue, if habituation condition is orange then put in this column the value of tot_time_oran
for index, row in flydata_AS_resume_df.iterrows():
    if row['habituation_cond'] == 'blue':
        flydata_AS_resume_df.at[index, 'tot_time_reinforc_obj'] = row['tot_time_blue']
    elif row['habituation_cond'] == 'orange':
        flydata_AS_resume_df.at[index, 'tot_time_reinforc_obj'] = row['tot_time_oran']
    else:
        flydata_AS_resume_df.at[index, 'tot_time_reinforc_obj'] = np.nan
        

In [76]:
flydata_AS_resume_df

,arena,exp,maze,starving_cond,habituation_cond,first_choice,tot_time_blue,tot_time_oran,tot_time_neutral,rate_interactions_blue,rate_interactions_oran,max_consec_time_blue,max_consec_time_oran,tot_time_reinforc_obj
0,2,5,0,starved,unused,blue,1964.70488,0.00000,1635.07608,1.000000,0.000000,958.26816,0.00000,NaN
104402,2,5,1,starved,unused,orange,251.60056,3101.51048,246.66992,0.276316,0.723684,63.23632,440.65440,NaN
208804,2,5,2,starved,unused,orange,517.82064,673.56680,2408.39352,0.368421,0.631579,139.26472,81.89000,NaN
313206,7,5,0,starved,orange,orange,325.52568,242.15304,3032.10224,0.709677,0.290323,53.65088,50.16840,242.15304
417608,7,5,1,starved,orange,orange,0.03448,2505.55816,1094.18832,0.000000,1.000000,0.03448,2371.43096,2505.55816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13572260,6,3,1,fed,orange,blue,506.71808,2178.17056,914.89232,0.344086,0.655914,76.71800,623.22600,2178.17056
13676662,6,3,2,fed,orange,blue,600.71056,2101.17672,897.89368,0.253521,0.746479,164.29720,827.07176,2101.17672
13781064,8,3,0,fed,orange,orange,487.09896,567.29944,2545.38256,0.490566,0.509434,24.20496,37.06600,567.29944
13885466,8,3,1,fed,orange,orange,864.89632,1687.72704,1047.15760,0.375000,0.625000,54.71976,30.68720,1687.72704


In [77]:
flydata_AS_df 

,index,head_x,head_y,thorax_x,thorax_y,abdo_x,abdo_y,time,arena,maze,exp,starving_cond,habituation_cond,pos_blue_obj,area
0,0,134.416687,443.169678,150.718246,443.773621,172.266769,446.496918,0.000000,2,0,5,starved,unused,left,neutral
1,1,134.481506,443.224609,153.064728,443.615417,172.361008,446.615692,0.034483,2,0,5,starved,unused,left,neutral
2,2,134.433365,443.181183,150.749207,443.730011,172.447113,446.680237,0.068966,2,0,5,starved,unused,left,neutral
3,3,134.386978,443.216705,150.719879,443.798920,172.404236,446.566772,0.103448,2,0,5,starved,unused,left,neutral
4,4,134.329590,440.829559,150.600082,443.740784,172.276642,446.632080,0.137931,2,0,5,starved,unused,left,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14094265,104397,105.827072,472.403839,112.086449,456.528503,118.310509,440.118469,3599.896552,8,2,3,fed,orange,left,neutral
14094266,104398,105.399048,472.236908,111.745148,456.459320,118.223831,440.161835,3599.931034,8,2,3,fed,orange,left,neutral
14094267,104399,105.769691,471.988434,111.970802,456.144928,118.426575,439.839447,3599.965517,8,2,3,fed,orange,left,neutral
14094268,104400,105.867027,469.476105,112.073944,455.944672,118.488075,437.254028,3600.000000,8,2,3,fed,orange,left,neutral


In [78]:
# Sqve the resume df to a CSV file
flydata_AS_resume_df.to_csv('flydata_AS_resume.csv', index=False)